# Store Slice as pickle

### Imports

In [1]:
from pathlib import Path
import numpy as np

from python_utils.utils import save_object
from python_utils.utils import load_object
from python_utils.data_processing import parse_tecplot_slice_to_dataframes_timeseries

### General Args

In [2]:
slice_folder_path = Path("../../../data/4pi_re1000_ha300.360_ch28.p1/SliceYZ_x0015/")

### Load

In [3]:
data_path = slice_folder_path / Path("sliceYZ_x0015_fld.dat")

In [4]:
data_frames = parse_tecplot_slice_to_dataframes_timeseries(data_path)

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\..\\data\\4pi_re1000_ha300.360_ch28.p1\\SliceYZ_x0015\\sliceYZ_x0015_fld.dat'

In [ ]:
# Example: Access first snapshot
print("N =", len(data_frames))
df_first_snapshot = data_frames[0]
n_x_y, n_v = df_first_snapshot.shape
n_y = n_z = np.sqrt(n_x_y)
print(f"n_y = {n_y}, n_z = {n_z}, n_v = {n_v}")
print(df_first_snapshot.head(n=5))

N = 0


IndexError: list index out of range

### Store

In [ ]:
pickle_dir = slice_folder_path / Path("pkl/sliceYZ_x0015_fld.pkl")

In [ ]:
save_object(data_frames, pickle_dir)

### Verification

In [ ]:
# Load the object back from the specified path
loaded_data = load_object(pickle_dir)

# Verify if it matches the original data
print("N =", len(loaded_data))
df_first_snapshot = loaded_data[0]
n_x_y, n_v = df_first_snapshot.shape
n_y = n_z = np.sqrt(n_x_y)
print(f"n_y = {n_y}, n_z = {n_z}, n_v = {n_v}")
print(df_first_snapshot.head(n=5))